In [ ]:
%pip install --upgrade pip
%pip install mem0ai pyautogen flaml

In [11]:
# Set up ENV Vars
import os

os.environ["OPENAI_API_KEY"] = "sk-xxx"

In [12]:
# AutoGen GPTAssistantAgent Capabilities:
# - Generates code based on user requirements and preferences.
# - Analyzes, refactors, and debugs existing code efficiently.
# - Maintains consistent coding standards across multiple sessions.
# - Remembers project-specific conventions and architectural decisions.
# - Learns from past interactions to improve future code suggestions.
# - Reduces repetitive explanations of coding preferences, enhancing productivity.
# - Adapts to team-specific practices for a more cohesive development process.

import logging
import os

from autogen import AssistantAgent, UserProxyAgent
from autogen.agentchat.contrib.gpt_assistant_agent import GPTAssistantAgent

logger = logging.getLogger(__name__)
logger.setLevel(logging.WARNING)

assistant_id = os.environ.get("ASSISTANT_ID", None)

# LLM Configuration
CACHE_SEED = 42  # choose your poison
llm_config = {
    "config_list": [{"model": "gpt-4o", "api_key": os.environ["OPENAI_API_KEY"]}],
    "cache_seed": CACHE_SEED,
    "timeout": 120,
    "temperature": 0.0,
}

assistant_config = {"assistant_id": assistant_id}

gpt_assistant = GPTAssistantAgent(
    name="assistant",
    instructions=AssistantAgent.DEFAULT_SYSTEM_MESSAGE,
    llm_config=llm_config,
    assistant_config=assistant_config,
)

user_proxy = UserProxyAgent(
    name="user_proxy",
    code_execution_config={
        "work_dir": "coding",
        "use_docker": False,
    },  # Please set use_docker=True if docker is available to run the generated code. Using docker is safer than running the generated code directly.
    is_termination_msg=lambda msg: "TERMINATE" in msg["content"],
    human_input_mode="NEVER",
    max_consecutive_auto_reply=1,
    llm_config=llm_config,
)

user_query = "Write a Python function that reverses a string."
# Initiate Chat w/o Memory
user_proxy.initiate_chat(gpt_assistant, message=user_query)

user_proxy (to assistant):

Write a Python function that reverses a string.

--------------------------------------------------------------------------------
assistant (to user_proxy):

Sure! Here is the Python code for a function that takes a string as input and returns the reversed string.

```python
def reverse_string(s):
    return s[::-1]

# Example usage
if __name__ == "__main__":
    example_string = "Hello, world!"
    reversed_string = reverse_string(example_string)
    print(f"Original string: {example_string}")
    print(f"Reversed string: {reversed_string}")
```

When you run this code, it will print the original string and the reversed string. You can replace `example_string` with any string you want to reverse.


--------------------------------------------------------------------------------

>>>>>>>> EXECUTING CODE BLOCK 0 (inferred language is python)...
user_proxy (to assistant):

exitcode: 0 (execution succeeded)
Code output: 
Original string: Hello, world!
Reversed 

ChatResult(chat_id=None, chat_history=[{'content': 'Write a Python function that reverses a string.', 'role': 'assistant', 'name': 'user_proxy'}, {'content': 'Sure! Here is the Python code for a function that takes a string as input and returns the reversed string.\n\n```python\ndef reverse_string(s):\n    return s[::-1]\n\n# Example usage\nif __name__ == "__main__":\n    example_string = "Hello, world!"\n    reversed_string = reverse_string(example_string)\n    print(f"Original string: {example_string}")\n    print(f"Reversed string: {reversed_string}")\n```\n\nWhen you run this code, it will print the original string and the reversed string. You can replace `example_string` with any string you want to reverse.\n', 'role': 'user', 'name': 'assistant'}, {'content': 'exitcode: 0 (execution succeeded)\nCode output: \nOriginal string: Hello, world!\nReversed string: !dlrow ,olleH\n', 'role': 'assistant', 'name': 'user_proxy'}, {'content': 'Great, the function worked as expected! The origi

In [16]:
# Benefits of Preference Memory in AutoGen Agents:
# - Personalization: Tailors responses to individual user or team preferences.
# - Consistency: Maintains uniform coding style and standards across sessions.
# - Efficiency: Reduces need to restate preferences, saving time in each interaction.
# - Adaptability: Evolves understanding of user needs over multiple conversations.
# - Context Retention: Keeps project-specific details accessible without repetition.
# - Improved Recommendations: Suggests solutions aligned with past preferences.
# - Long-term Learning: Accumulates knowledge to enhance future interactions.
# - Reduced Cognitive Load: Users don't need to remember and restate all preferences.


# Setting memory (preference) for the user
from mem0 import Memory

# Initialize Mem0
MEM0_MEMORY_CLIENT = Memory()

USER_ID = "chicory.ai.user"
MEMORY_DATA = """
* Preference for readability: The user prefers code to be explicitly written with clear variable names.
* Preference for comments: The user prefers comments explaining each step.
* Naming convention: The user prefers camelCase for variable names.
* Docstrings: The user prefers functions to have a descriptive docstring.
"""
AGENT_ID = "chicory.ai"

# Add preference data to memory
MEM0_MEMORY_CLIENT.add(MEMORY_DATA, user_id=USER_ID)
MEM0_MEMORY_CLIENT.add(MEMORY_DATA, agent_id=AGENT_ID)

/var/folders/z6/3w4ng1lj3mn4vmhplgc4y0580000gn/T/ipykernel_77647/3850691550.py:28: DeprecationWarning: The current add API output format is deprecated. To use the latest format, set `api_version='v1.1'`. The current format will be removed in mem0ai 1.1.0 and later versions.
  MEM0_MEMORY_CLIENT.add(MEMORY_DATA, user_id=USER_ID)
/var/folders/z6/3w4ng1lj3mn4vmhplgc4y0580000gn/T/ipykernel_77647/3850691550.py:29: DeprecationWarning: The current add API output format is deprecated. To use the latest format, set `api_version='v1.1'`. The current format will be removed in mem0ai 1.1.0 and later versions.
  MEM0_MEMORY_CLIENT.add(MEMORY_DATA, agent_id=AGENT_ID)


{'message': 'ok'}

## Option 1: 
Using Direct Prompt Injection:
`user memory example`

In [17]:
# Retrieve the memory
relevant_memories = MEM0_MEMORY_CLIENT.search(user_query, user_id=USER_ID, limit=3)
relevant_memories_text = "\n".join(mem["memory"] for mem in relevant_memories)
print("Relevant memories:")
print(relevant_memories_text)

prompt = f"{user_query}\n Coding Preferences: \n{relevant_memories_text}"
browse_result = user_proxy.initiate_chat(gpt_assistant, message=prompt)

/var/folders/z6/3w4ng1lj3mn4vmhplgc4y0580000gn/T/ipykernel_77647/703598432.py:2: DeprecationWarning: The current get_all API output format is deprecated. To use the latest format, set `api_version='v1.1'`. The current format will be removed in mem0ai 1.1.0 and later versions.
  relevant_memories = MEM0_MEMORY_CLIENT.search(user_query, user_id=USER_ID, limit=3)
INFO:autogen.agentchat.contrib.gpt_assistant_agent:Clearing thread thread_BOgA5TdAOrYqSHLVpxc5ZifB


Relevant memories:
Prefers functions to have a descriptive docstring
Prefers camelCase for variable names
Prefers code to be explicitly written with clear variable names
user_proxy (to assistant):

Write a Python function that reverses a string.
 Coding Preferences: 
Prefers functions to have a descriptive docstring
Prefers camelCase for variable names
Prefers code to be explicitly written with clear variable names

--------------------------------------------------------------------------------
assistant (to user_proxy):

Sure, I will write a Python function that reverses a given string with clear and descriptive variable names, along with a descriptive docstring.

```python
def reverseString(inputString):
    """
    Reverses the given string.

    Parameters:
    inputString (str): The string to be reversed.

    Returns:
    str: The reversed string.
    """
    # Initialize an empty string to store the reversed version
    reversedString = ""

    # Iterate through each character 

## Option 2:
Using UserProxyAgent: 
`agent memory example`

In [18]:
# UserProxyAgent in AutoGen:
# - Acts as intermediary between humans and AI agents in the AutoGen framework.
# - Simulates user behavior and interactions within multi-agent conversations.
# - Can be configured to execute code blocks received in messages.
# - Supports flexible human input modes (e.g., ALWAYS, TERMINATE, NEVER).
# - Customizable for specific interaction patterns and behaviors.
# - Can be integrated with memory systems like mem0 for enhanced functionality.
# - Capable of fetching relevant memories before processing a query.
# - Enables more context-aware and personalized agent responses.
# - Bridges the gap between human input and AI processing in complex workflows.


class Mem0ProxyCoderAgent(UserProxyAgent):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.memory = MEM0_MEMORY_CLIENT
        self.agent_id = kwargs.get("name")

    def initiate_chat(self, assistant, message):
        # Retrieve memory for the agent
        agent_memories = self.memory.search(message, agent_id=self.agent_id, limit=3)
        agent_memories_txt = "\n".join(mem["memory"] for mem in agent_memories)
        prompt = f"{message}\n Coding Preferences: \n{str(agent_memories_txt)}"
        response = super().initiate_chat(assistant, message=prompt)
        # Add new memory after processing the message
        response_dist = response.__dict__ if not isinstance(response, dict) else response
        MEMORY_DATA = [{"role": "user", "content": message}, {"role": "assistant", "content": response_dist}]
        self.memory.add(MEMORY_DATA, agent_id=self.agent_id)
        return response

In [19]:
mem0_user_proxy = Mem0ProxyCoderAgent(
    name=AGENT_ID,
    code_execution_config={
        "work_dir": "coding",
        "use_docker": False,
    },  # Please set use_docker=True if docker is available to run the generated code. Using docker is safer than running the generated code directly.
    is_termination_msg=lambda msg: "TERMINATE" in msg["content"],
    human_input_mode="NEVER",
    max_consecutive_auto_reply=1,
)
code_result = mem0_user_proxy.initiate_chat(gpt_assistant, message=user_query)

chicory.ai (to assistant):

Write a Python function that reverses a string.
 Coding Preferences: 
Prefers functions to have a descriptive docstring
Prefers camelCase for variable names
Prefers code to be explicitly written with clear variable names

--------------------------------------------------------------------------------


/var/folders/z6/3w4ng1lj3mn4vmhplgc4y0580000gn/T/ipykernel_77647/1070513538.py:13: DeprecationWarning: The current get_all API output format is deprecated. To use the latest format, set `api_version='v1.1'`. The current format will be removed in mem0ai 1.1.0 and later versions.
  agent_memories = self.memory.search(message, agent_id=self.agent_id, limit=3)


assistant (to chicory.ai):

Sure, I'll write a Python function that reverses a string following your coding preferences.

```python
def reverseString(inputString):
    """
    Reverse the given string.

    Parameters:
    inputString (str): The string to be reversed.

    Returns:
    str: The reversed string.
    """
    reversedString = inputString[::-1]
    return reversedString

# Example usage:
inputString = "hello"
print(reverseString(inputString))  # Output: "olleh"
```

This function `reverseString` takes an `inputString`, reverses it using slicing (`inputString[::-1]`), and returns the reversed string. The docstring provides a clear description of the function's purpose, parameters, and return value. The variable names are explicitly descriptive.


--------------------------------------------------------------------------------

>>>>>>>> EXECUTING CODE BLOCK 0 (inferred language is python)...
chicory.ai (to assistant):

exitcode: 0 (execution succeeded)
Code output: 
olleh




/var/folders/z6/3w4ng1lj3mn4vmhplgc4y0580000gn/T/ipykernel_77647/1070513538.py:20: DeprecationWarning: The current add API output format is deprecated. To use the latest format, set `api_version='v1.1'`. The current format will be removed in mem0ai 1.1.0 and later versions.
  self.memory.add(MEMORY_DATA, agent_id=self.agent_id)


# Option 3:
Using Teachability:
`agent memory example`

In [20]:
# building on top of existing Teachability package from autogen
# from autogen.agentchat.contrib.capabilities.teachability import Teachability

# AutoGen Teachability Feature:
# - Enables agents to learn and remember across multiple chat sessions.
# - Addresses the limitation of traditional LLMs forgetting after conversations end.
# - Uses vector database to store "memos" of taught information.
# - Can remember facts, preferences, and even complex skills.
# - Allows for cumulative learning and knowledge retention over time.
# - Enhances personalization and adaptability of AI assistants.
# - Can be integrated with mem0 for improved memory management.
# - Potential for more efficient and context-aware information retrieval.
# - Enables creation of AI agents with long-term memory and learning abilities.
# - Improves consistency and reduces repetition in user-agent interactions.

from cookbooks.helper.mem0_teachability import Mem0Teachability

teachability = Mem0Teachability(
    verbosity=2,  # for visibility of what's happening
    recall_threshold=0.5,
    reset_db=False,  # Use True to force-reset the memo DB, and False to use an existing DB.
    agent_id=AGENT_ID,
    memory_client=MEM0_MEMORY_CLIENT,
)
teachability.add_to_agent(user_proxy)

In [21]:
# Initiate Chat w/ Teachability + Memory
user_proxy.initiate_chat(gpt_assistant, message=user_query)

INFO:autogen.agentchat.contrib.gpt_assistant_agent:Clearing thread thread_dfnrEoXX4MoZesb0cerO9LKm


user_proxy (to assistant):

Write a Python function that reverses a string.

--------------------------------------------------------------------------------
assistant (to user_proxy):

Sure, I'll provide you with a Python function that takes a string as input and returns the reversed string. Here is the complete code:

```python
# filename: reverse_string.py

def reverse_string(s: str) -> str:
    """
    This function takes a string as input and returns the reversed string.
    
    :param s: Input string to be reversed
    :return: Reversed string
    """
    return s[::-1]

# Example usage
input_string = "Hello, World!"
reversed_string = reverse_string(input_string)
print(f"Original string: {input_string}")
print(f"Reversed string: {reversed_string}")
```

Save the above code in a file named `reverse_string.py`, then execute it. This script defines the `reverse_string` function and demonstrates its usage by reversing the string "Hello, World!". It will print both the original and r

ChatResult(chat_id=None, chat_history=[{'content': 'Write a Python function that reverses a string.', 'role': 'assistant', 'name': 'user_proxy'}, {'content': 'Sure, I\'ll provide you with a Python function that takes a string as input and returns the reversed string. Here is the complete code:\n\n```python\n# filename: reverse_string.py\n\ndef reverse_string(s: str) -> str:\n    """\n    This function takes a string as input and returns the reversed string.\n    \n    :param s: Input string to be reversed\n    :return: Reversed string\n    """\n    return s[::-1]\n\n# Example usage\ninput_string = "Hello, World!"\nreversed_string = reverse_string(input_string)\nprint(f"Original string: {input_string}")\nprint(f"Reversed string: {reversed_string}")\n```\n\nSave the above code in a file named `reverse_string.py`, then execute it. This script defines the `reverse_string` function and demonstrates its usage by reversing the string "Hello, World!". It will print both the original and revers